# A Custom Multi Layer Perceptron For Time Series Classification

### Importing Three Different Time Series Dataset by using sktime's `load_UCR_UEA_dataset` 
#### About the datasets:
* First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.
* Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.
* Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [ ]:
# ------ Importing necessary packages!
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [3]:
# ------ Importing the datasets!
# 2 Classes, 1D, Image
X_train_1, y_train_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="train", return_type="numpy2D")
X_test_1, y_test_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="test", return_type="numpy2D")

# 2 Classes, 1D, Sensor 
X_train_2, y_train_2 = load_UCR_UEA_dataset(name= "ItalyPowerDemand", split="train", return_type="numpy2D")
X_test_2, y_test_2 = load_UCR_UEA_dataset(name="ItalyPowerDemand", split="test", return_type="numpy2D")

# 3Classes, 1D, Simulated
X_train_3, y_train_3 = load_UCR_UEA_dataset(name = "BME", split="train", return_type="numpy2D")
X_test_3, y_test_3 = load_UCR_UEA_dataset(name = "BME", split="test", return_type="numpy2D")

In [6]:
print("Dataset 1:", X_train_1.shape, X_test_1.shape)
print("Dataset 2:", X_train_2.shape, X_test_2.shape)
print("Dataset 3:", X_train_3.shape, X_test_3.shape)

Dataset 1: (600, 80) (276, 80)
Dataset 2: (67, 24) (1029, 24)
Dataset 3: (30, 128) (150, 128)


We have different datasets, which means we must have a function that can take in different datasets with different length.

In [ ]:
def create_mlp_model(input_shape, num_classes):
    mlp_model = tf.keras.Sequential([
        keras.layers.Dropout(0.1, input_shape= input_shape),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    mlp_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=0.95, epsilon=1e-8),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return mlp_model